In [ ]:
# Tải mô hình phát hiện khuôn
!pip install face_recognition
import face_recognition


  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached face_recognition_models-0.3.0.tar.gz (100.1 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=156b331755acb996574fc0eaea66385376550cbd66344411be8a9776014b9fa8
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


In [ ]:
import os
import cv2
import random

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.utils import to_categorical
from tensorflow.keras.models import load_model


In [ ]:
IMG_SIZE = [224, 224, 3]  # kích thước đầu vào

PIXEL_MEAN = [103.939, 116.779, 123.68] # BGR pixel mean value

NUM_CLASSES = 2           # số lượng lớp đầu ra

BKG_LABEL = 0             #

LEARNING_RATE = 0.001     # tốc độ học - xác định tốc độ cập nhật các trọng số trong quá trình tối ưu hóa mô hình.

DECAY_RATE = 0.95         # Tốc độ học sẽ bị giảm sau mốc sự kiện nhất định

NUM_EPOCHS = 20           # Một epoch đại diện cho một lần model đã được huấn luyện trên tất cả các mẫu trong tập dữ liệu huấn luyện

TRAIN_BATCH_SIZE = 64     # Dữ liệu huấn luyện thường được chia thành các batch (nhóm) để tăng tốc độ huấn luyện và tối ưu hóa


MODEL_DIR_PREFIX = 'ckpt' #

MODEL_NAME = 'model'      #

TEST_BATCH_SIZE = 8       #

In [ ]:
base_model = tf.keras.applications.VGG16(
    weights=None,             # Mô hình chưa được đào tạo trước
    include_top=False,        # base_model không bao gồm các lớp Fully Connected
    input_shape=IMG_SIZE
)


# Thêm các tầng phân loại mới
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
    # ngẫu nhiên "loại bỏ" một phần các đơn vị đầu vào trong quá trình huấn luyện
    # giảm hiện tượng overfitting
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'))


In [ ]:

# Sau decay_steps, giảm tỷ lệ học một lần
decay_steps = 1000

# Tạo lịch trình học tập tùy chỉnh
learning_rate_schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[decay_steps],
    values=[LEARNING_RATE, LEARNING_RATE * DECAY_RATE]
)

# Xác định trình tối ưu hóa với lịch trình học tập tùy chỉnh
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate_schedule, momentum=0.9)

# Compile mô hình
model.compile(optimizer=optimizer,
              # Tính toán sự khác biệt giữa xác xuất dự đoán và thực tế
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Chuẩn bị dữ liệu huấn luyện
def convert_positive_to_negative_example(image_path):
    image = cv2.imread(image_path)
    # Dùng face_recognition để phát hiện khuôn mặt và các điểm đặc trưng
    face_landmarks_list = face_recognition.face_landmarks(image)

# Kiểm tra xem có khuôn mặt nào được phát hiện hay không
    result = image
    for i in range(0, len(face_landmarks_list)):
        # Lấy danh sách các điểm đặc trưng của khuôn mặt đầu tiên
        face_landmarks = face_landmarks_list[i]

        # Lấy các điểm đặc trưng của khuôn mặt
        eyebrow_left = face_landmarks['left_eyebrow']
        eyebrow_right = face_landmarks['right_eyebrow']
        eye_left = face_landmarks['left_eye']
        eye_right = face_landmarks['right_eye']
        nose = face_landmarks['nose_tip']
        lip = face_landmarks['bottom_lip']

    # cắt hình
        # Tính toán tọa độ góc trái trên của hình ảnh mới
        top_left_x = min(eyebrow_left[0][0], eye_left[0][0], nose[0][0], lip[6][0])
        top_left_y = image.shape[:2][1]
        for i in range(0, 5):
            top_left_y = min(top_left_y, eyebrow_left[i][1], eyebrow_right[i][1])

        # Tính toán tọa độ góc phải dưới của hình ảnh mới
        bottom_right_x = max(eyebrow_right[-1][0], eye_right[3][0], nose[0][0], lip[0][0])
        bottom_right_y = 0
        for i in range(0, 12):
            bottom_right_y = max(bottom_right_y, lip[i][1])

        #xét điều kiện tránh cắt ra ngoài ảnh
        top_left_x = max(top_left_x, 0)
        top_left_y = max(top_left_y, 0)
        bottom_right_x = min(bottom_right_x, image.shape[:2][0])
        bottom_right_y = min(bottom_right_y, image.shape[:2][1])

        cropped_image = image[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
        cropped_image_size = cropped_image.shape[:2]

        # Phóng to hoặc thu nhỏ ảnh ngẫu nhiên
        scale_factor = random.uniform(0.5, 1.5)  # Tính toán tỷ lệ ngẫu nhiên
        resized_image = cv2.resize(cropped_image, None, fx=scale_factor, fy=scale_factor)

        # Ghép lại ảnh ban đầu
        resized_image = cv2.resize(resized_image, (cropped_image_size[1], cropped_image_size[0])) # Trả lại kích thước ban đầu cho ảnh đã cắt
        image[top_left_y:bottom_right_y, top_left_x:bottom_right_x] = resized_image  # Đặt ảnh đã bị cắt vào vị trí ảnh cắt lúc đầu trong ảnh gốc

    #Áp dụng làm mờ cho vùng mặt nạ được vẽ gồm lông mày và môi dưới
        # Tạo mặt nạ
        mask_points = np.concatenate((eyebrow_right[-1], lip[0], lip[1], lip[2], lip[3], lip[4], lip[5], lip[6], eyebrow_left[0]), axis=0).reshape(-1, 2)

        for i in range(1, 5):
            if eyebrow_left[i-1][1] > eyebrow_left[i][1]:
                mask_points = np.concatenate((mask_points, np.expand_dims(eyebrow_left[i], axis=0)), axis=0)

        for i in range(1, 5):
            if eyebrow_right[-i][1] > eyebrow_right[-i-1][1]:
                mask_points = np.concatenate((np.expand_dims(eyebrow_right[-i-1], axis=0), mask_points), axis=0)

        # Tạo mask ban đầu
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        cv2.fillPoly(mask, [mask_points.astype(np.int32)], 255)

        # Mở rộng mask để cắt thêm phần xung quanh
        kernel_size = 5  # Kích thước kernel dilation
        border_size = 10  # Kích thước viền
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
        expanded_mask = cv2.dilate(mask, kernel, iterations=border_size)

        # Cắt vùng đã chọn từ hình ảnh gốc
        face_cut = cv2.bitwise_and(image, image, mask=expanded_mask)

        # Áp dụng hiệu ứng làm mờ Gaussian cho vùng đã được cắt ra
        blurred_face = cv2.GaussianBlur(face_cut, (5, 5), 0)

        mask = np.expand_dims(mask, axis=2)
        result = np.where(mask == 0, image, blurred_face)
    return result


In [ ]:
#tạo ra batch

def batch_generator(data_dir, batch_size):
    file_list = os.listdir(data_dir)
    num_samples = len(file_list)
    num_batches = num_samples // batch_size
    batches = []
    def generator():
        for batch_index in range(num_batches):
            start_index = batch_index * batch_size
            end_index = (batch_index + 1) * batch_size
            batch_files = file_list[start_index:end_index]

            x_batch = []
            y_batch = []

            # Lấy ngẫu nhiên nửa số lượng ví dụ tích cực
            positive_examples = np.random.choice(batch_files, size=batch_size//2, replace=False)

            for file_name in batch_files:
                # Đường dẫn đầy đủ của hình ảnh
                image_path = os.path.join(data_dir, file_name)
                image = cv2.imread(image_path)

                if file_name not in positive_examples and len(face_recognition.face_locations(image)) > 0:
                    image = convert_positive_to_negative_example(image_path)
                    # Tiền xử lý ảnh
                    # Thay đổi kích thước ảnh
                    image = cv2.resize(image, (224, 224))

                    # Chuẩn hóa pixel
                    image -= np.array(PIXEL_MEAN, dtype=np.uint8)
                    label = 1
                    x_batch.append(image)
                    y_batch.append(label)
                else :
                    # Tiền xử lý ảnh
                    # Thay đổi kích thước ảnh
                    image = cv2.resize(image, (224, 224))

                    # Chuẩn hóa pixel
                    image -= np.array(PIXEL_MEAN, dtype=np.uint8)
                    label = 0
                    x_batch.append(image)
                    y_batch.append(label)

            x_batch = np.array(x_batch)
            y_batch = tf.keras.utils.to_categorical(y_batch, num_classes=2)

            yield x_batch, y_batch

    dataset = tf.data.Dataset.from_generator(generator, output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.uint8),
        tf.TensorSpec(shape=(None, 2), dtype=tf.float32)
    ))

    return dataset
# Tạo dataset cho đánh giá
eval_data_dir = "/content/drive/MyDrive/eval_data"
eval_dataset = batch_generator(eval_data_dir, TEST_BATCH_SIZE)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             514 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,137,986 (80.64 MB)

 Trainable params: 21,137,986 (80.64 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
eval_data_dir = "/content/drive/MyDrive/eval_data"
eval_dataset = batch_generator(eval_data_dir, 8)

batch_size = 64
data_dir = "/content/drive/MyDrive/positive_example"

num_epochs = 10
batch_size = 64
loss_values = []
accuracy_values = []
best_accuracy = 0.0  # Biến để theo dõi độ chính xác tốt nhất

for epoch in range(num_epochs):
    for x_batch, y_batch in batch_generator(data_dir, batch_size):
        model.train_on_batch(x_batch, y_batch)

    # Đánh giá mô hình trên tập validation
    val_loss, val_accuracy = model.evaluate(eval_dataset)

    loss_values.append(val_loss)
    accuracy_values.append(val_accuracy)

    # In thông tin về epoch
    print(f"Epoch {epoch+1}/{num_epochs} - val_loss: {val_loss:.4f} - val_accuracy: {val_accuracy:.4f}")

    # Kiểm tra nếu độ chính xác hiện tại tốt hơn độ chính xác tốt nhất
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        # Lưu lại trạng thái của mô hình
        model.save_weights("/content/drive/MyDrive/best_model.h5")
        model.save("/content/drive/MyDrive/Model.h5")


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: __call__(): incompatible function arguments. The following argument types are supported:
    1. (self: _dlib_pybind11.fhog_object_detector, image: numpy.ndarray, upsample_num_times: int = 0) -> _dlib_pybind11.rectangles

Invoked with: <_dlib_pybind11.fhog_object_detector object at 0x7a6eecce9cf0>, None, 1
Traceback (most recent call last):

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(itera

In [ ]:
# Đường dẫn dữ liệu đánh giá và huấn luyện
train_data_dir = "/content/drive/My Drive/dataset_evaluation/Dataset/Train"


# Huấn luyện mô hình
num_epochs = 20
loss_values = []
accuracy_values = []
best_accuracy = 0.0  # Biến để theo dõi độ chính xác tốt nhất

for epoch in range(num_epochs):
    for x_batch, y_batch in batch_generator(train_data_dir, TRAIN_BATCH_SIZE):
        model.train_on_batch(x_batch, y_batch)

    # Đánh giá mô hình trên tập validation
    val_loss, val_accuracy = model.evaluate(eval_dataset)

    loss_values.append(val_loss)
    accuracy_values.append(val_accuracy)

    # In thông tin về epoch
    print(f"Epoch {epoch+1}/{num_epochs} - val_loss: {val_loss:.4f} - val_accuracy: {val_accuracy:.4f}")

    # Kiểm tra nếu độ chính xác hiện tại tốt hơn độ chính xác tốt nhất
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        # Lưu lại trạng thái của mô hình
        model.save("/content/drive/MyDrive/Model_23012025.h5")

50/50 [==============================] - 36s 522ms/step - loss: 0.0583 - accuracy: 0.9825
Epoch 1/20 - val_loss: 0.0583 - val_accuracy: 0.9825
50/50 [==============================] - 27s 534ms/step - loss: 0.0507 - accuracy: 0.9800
Epoch 2/20 - val_loss: 0.0507 - val_accuracy: 0.9800
50/50 [==============================] - 28s 543ms/step - loss: 0.0764 - accuracy: 0.9700
Epoch 3/20 - val_loss: 0.0764 - val_accuracy: 0.9700
50/50 [==============================] - 25s 498ms/step - loss: 0.0813 - accuracy: 0.9700
Epoch 4/20 - val_loss: 0.0813 - val_accuracy: 0.9700
50/50 [==============================] - 25s 502ms/step - loss: 0.0628 - accuracy: 0.9800
Epoch 5/20 - val_loss: 0.0628 - val_accuracy: 0.9800
50/50 [==============================] - 25s 494ms/step - loss: 0.0735 - accuracy: 0.9700
Epoch 6/20 - val_loss: 0.0735 - val_accuracy: 0.9700
50/50 [==============================] - 24s 482ms/step - loss: 0.0895 - accuracy: 0.9750
Epoch 7/20 - val_loss: 0.0895 - val_accuracy: 0.9750

In [ ]:
# Vẽ biểu đồ loss của train và validation
plt.plot(range(1, 2*num_epochs+1), loss_values, label='Train Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()
plt.show()

# Vẽ biểu đồ độ chính xác của train và validation
plt.plot(range(1, 2*num_epochs+1), accuracy_values, label='Train Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()
plt.show()

# Nháp


In [ ]:
# Tải mô hình đã huấn luyện nếu có
model_path = '/content/drive/MyDrive/Model.h5'
if os.path.exists(model_path):
    model = load_model(model_path)

# Đường dẫn dữ liệu đánh giá và huấn luyện
train_data_dir = "/content/drive/My Drive/dataset_evaluation/Dataset/Train"


# Huấn luyện mô hình
num_epochs = 20
loss_values = []
accuracy_values = []
best_accuracy = 0.0  # Biến để theo dõi độ chính xác tốt nhất

for epoch in range(num_epochs):
    for x_batch, y_batch in batch_generator(train_data_dir, TRAIN_BATCH_SIZE):
        model.train_on_batch(x_batch, y_batch)

    # Đánh giá mô hình trên tập validation
    val_loss, val_accuracy = model.evaluate(eval_dataset)

    loss_values.append(val_loss)
    accuracy_values.append(val_accuracy)

    # In thông tin về epoch
    print(f"Epoch {epoch+1}/{num_epochs} - val_loss: {val_loss:.4f} - val_accuracy: {val_accuracy:.4f}")
    model.save_weights("/content/drive/MyDrive/best_model.h5")

50/50 [==============================] - 25s 491ms/step - loss: 0.0656 - accuracy: 0.9775
Epoch 1/20 - val_loss: 0.0656 - val_accuracy: 0.9775
50/50 [==============================] - 24s 478ms/step - loss: 0.0666 - accuracy: 0.9750
Epoch 2/20 - val_loss: 0.0666 - val_accuracy: 0.9750
50/50 [==============================] - 25s 491ms/step - loss: 0.0622 - accuracy: 0.9825
Epoch 3/20 - val_loss: 0.0622 - val_accuracy: 0.9825
50/50 [==============================] - 24s 479ms/step - loss: 0.0840 - accuracy: 0.9700
Epoch 4/20 - val_loss: 0.0840 - val_accuracy: 0.9700
50/50 [==============================] - 26s 528ms/step - loss: 0.0686 - accuracy: 0.9725
Epoch 5/20 - val_loss: 0.0686 - val_accuracy: 0.9725
50/50 [==============================] - 25s 502ms/step - loss: 0.0801 - accuracy: 0.9700
Epoch 6/20 - val_loss: 0.0801 - val_accuracy: 0.9700
50/50 [==============================] - 25s 493ms/step - loss: 0.0578 - accuracy: 0.9800
Epoch 7/20 - val_loss: 0.0578 - val_accuracy: 0.9800

In [ ]:
!unzip /content/inpainting.zip -d /content/data_test

!unzip /content/82.zip -d /content/82



In [ ]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Đường dẫn thư mục và mô hình
folder_path = '/content/data_test/25'

# Hàm tiền xử lý ảnh
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)  # Resize ảnh
    img_array = img_to_array(img)  # Chuyển đổi thành mảng NumPy
    img_array = np.expand_dims(img_array, axis=0)  # Thêm chiều batch
    img_array = img_array / 255.0  # Chuẩn hóa giá trị pixel về [0, 1]
    return img_array

# Hàm dự đoán và tính accuracy
def predict_from_folder(model, folder_path):
    count_is_deepfake = 0
    total_files = 0

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        # Kiểm tra xem tệp có phải là ảnh không
        if file_name.endswith(('.jpg', '.jpeg', '.png')):
            total_files += 1
            img_array = preprocess_image(file_path)  # Tiền xử lý ảnh
            pred = model.predict(img_array)  # Dự đoán
            label = np.argmax(pred, axis=1)  # Lấy nhãn (0 hoặc 1)
            if label == 1:
                count_is_deepfake += 1

    # Tính accuracy
    if total_files > 0:
        accuracy = count_is_deepfake / total_files
        print(f"Accuracy: {accuracy * 100:.2f}%")
    else:
        print("Không tìm thấy ảnh trong thư mục.")

# # Gọi hàm dự đoán
# predict_from_folder(model, folder_path)


In [ ]:
# Đường dẫn thư mục và mô hình
folder_path = '/content/82/82'
# Gọi hàm dự đoán
predict_from_folder(model, folder_path)


In [ ]:

file_path = '/content/Aaron_Eckhart_0001.jpg'
img_array = preprocess_image(file_path)  # Tiền xử lý ảnh
pred = model.predict(img_array)  # Dự đoán
print(pred)
label = np.argmax(pred, axis=1)  # Lấy nhãn (0 hoặc 1)
if label == 1:
    print("is_deepfake")
else:
    print("not_deepfake")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[[0.49979946 0.5002005 ]]
is_deepfake
